In [53]:
import pandas as pd
import numpy as np
import pickle
import re
import os
from os.path import join
from tqdm import tqdm

In [54]:
##################################
company = "대성"
basedate = "230904"
filename_raw = "계좌조회새창_20230904_1955"
##################################
filename_new = "2."+company+"_전체_원데이터_"+basedate
wd = join(r"D:\3.자산\전산 dataset", company, basedate)
# 계좌 원데이터
acc_raw = pd.read_pickle(join(wd, filename_raw + ".pkl"))
# 자산정리 원데이터
acc_new = pd.read_pickle(join(wd, filename_new + ".pkl"))

In [55]:
# 원데이터 가공 
raw = acc_raw.copy()
# 1) 시효중단은 시효완성월을 중단으로 하기
raw["시효완성월"] = np.where(raw.시효중단여부=="Y", "중단", raw["시효완성월"])
# 2) 필요칼럼만 : "시효중단여부"은 이제 필요 x
raw = raw[["계좌키", "시효완성월", "시효완성일", "시효연장사유"]].rename(columns={"시효연장사유":"시효사유"})
# 3) 시효 임박건
raw = raw[raw.시효완성월.isin(["시효만료", "당월도래", "1개월", "2개월"])]

In [56]:
# 자산정리파일 가공
# 1) 종료, 조정건 제외
new = acc_new[acc_new.새채무상태.isin(["개인", "법인", "보증인정상", "시효", "종료처리요망"])].copy()
# 2) 필요 컬럼 선택
new = new[["새채무자키", "채무자키", "계좌키", "성명", "시효완성월", "시효완성일", "시효사유", "새채무상태", "채무상태", "담당자", "새보증인상태", "원채권사", "매각사구분"]]
print(len(raw), len(new))

In [58]:
# 합치기 new에서 종료건과 조정건은 제외됐고(모든 무담보), raw에서 (시효임박건)을 추렸다. inner : 무담보 중 시효임박
merged = pd.merge(new, raw, on='계좌키', how='inner', suffixes=["_확장", "_전산"])

# 칼럼순서 변경
merged = merged.reindex(columns=['새채무자키', '채무자키', '계좌키', '성명', '시효완성월_전산', '시효완성일_전산', '시효사유_전산','시효완성월_확장', '시효완성일_확장', '시효사유_확장', '채권상태',
       '새채무상태', '채무상태', '담당자', '새보증인상태', '원채권사', '매각사구분'])
# 칼럼명에서 시효 빼기
merged = merged.rename(columns={'시효완성월_전산':'완성월_전산', '시효완성일_전산':'완성일_전산', '시효사유_전산':'사유_전산','시효완성월_확장':'완성월_확장', '시효완성일_확장':'완성일_확장', '시효사유_확장':'사유_확장'})
# 정렬
merged.sort_values(['새채무자키','계좌키','시효완성일_전산'], inplace=True)

# 그룹화 : 그룹순서대로 정렬된다. 그룹화후에 시효완성일로 정렬하더라도 결과는 같다.
#result = merged.groupby('새채무자키')

In [59]:
result = merged.set_index(['새채무자키','채무자키'],drop=True).copy()
result.to_excel(join(r"D:\4.관리\시효관리", company+"_시효임박건_"+basedate+".xlsx"))

In [ ]:
# 그룹화된 항목 필터링 되게 엑셀 편집하기
# 그룹화된 범위 전체 복사 > 빈셀에 붙여넣기
# 그룹화된 범위 셀병합 해제, 그대로 f5 > 빈칸선택 > =위행 > ctrl+enter(일괄적용)
# 붙여넣은 그룹화된 범위1 복사 > 병합 해제한 해당 범위 서식붙여넣기 
# 붙여넣은 그룹화된 범위2 복사 > 병합 해제한 해당 범위 서식붙여넣기 